# Amazon Bedrock AgentCore Runtime で Strands Agent を OpenLIT Observability と共に使用

## 概要

このノートブックでは、OpenLIT 可観測性統合を使用して Strands エージェントを Amazon Bedrock AgentCore Runtime にデプロイする方法を説明します。この実装は Amazon Bedrock Claude モデルを使用し、OpenTelemetry (OTEL) を通じてテレメトリデータを [OpenLIT](https://github.com/openlit/openlit) に送信します。

## 主なコンポーネント

- **Strands Agents**: 組み込みテレメトリサポートを持つ LLM 駆動エージェントを構築するための Python フレームワーク
- **Amazon Bedrock AgentCore Runtime**: AWS でエージェントをホスティングおよびスケーリングするためのマネージドランタイムサービス
- **OpenLIT**: OpenTelemetry 上に構築された LLM アプリケーションと AI エージェント向けオープンソース可観測性プラットフォーム
- **OpenTelemetry**: テレメトリデータの収集とエクスポートのための業界標準プロトコル

## アーキテクチャ

エージェントはコンテナ化され、AgentCore Runtime にデプロイされ、呼び出し用の HTTP エンドポイントを提供します。テレメトリデータは Strands エージェントから OTEL エクスポーターを通じて OpenLIT に流れ、監視とデバッグに使用されます。この実装では、代わりに OpenLIT を使用するために AgentCore のデフォルトの可観測性を無効にしています。

## 前提条件

- Python 3.10+
- Bedrock および AgentCore 権限を持つ AWS 認証情報
- [OpenLIT](https://github.com/openlit/openlit) がデプロイされていること
- Docker がローカルにインストールされていること
- us-west-2 で Amazon Bedrock Claude モデルへのアクセス

## OpenLIT セットアップ

エージェントのデプロイを進める前に、テレメトリデータを受信するために OpenLIT をセットアップする必要があります。OpenLIT は Amazon Bedrock AgentCore Runtime からアクセス可能である必要があります。

### デプロイオプション

OpenLIT をデプロイするには、主に 2 つのオプションがあります：

#### オプション 1: Docker デプロイ（テスト用に最速）
Docker Compose を使用して OpenLIT をデプロイします。これは開始するための最も簡単なアプローチです：

```bash
# Docker Compose を使用（クイックセットアップに推奨）
git clone https://github.com/openlit/openlit.git
cd openlit
docker compose up -d
```

これにより、`http://localhost:3000`（UI）と `http://localhost:4318`（OTEL エンドポイント）で OpenLIT が起動します。

AgentCore がアクセスできるようにするには、AgentCore が到達できるマシン（例：EC2 インスタンス）にデプロイする必要があります：
1. パブリック IP を持つ EC2 インスタンスまたはコンテナサービスにデプロイ
2. ポート 4318 がアクセス可能であることを確認（インバウンドトラフィックを許可するようにセキュリティグループを設定）
3. AgentCore 設定でパブリックエンドポイント URL（例：`http://<ec2-public-ip>:4318`）を使用

#### オプション 2: Kubernetes デプロイ（本番対応）
本番環境では、Helm を使用して Kubernetes に OpenLIT をデプロイします：

```bash
# OpenLIT Helm リポジトリを追加
helm repo add openlit https://openlit.github.io/helm-charts
helm repo update

# OpenLIT をインストール
helm install openlit openlit/openlit
```

**デフォルト設定：**
- デフォルトでは、OpenLIT はパブリック IP を持つ LoadBalancer サービスを作成します
- これにより、OTEL エンドポイントは `http://<load-balancer-ip>:4318` でパブリックにアクセス可能になります
- UI は `http://<load-balancer-ip>:3000` でアクセス可能になります

**VPC/プライベート設定：**
OpenLIT を VPC 内でプライベートに保ちたい場合（本番環境に推奨）：

1. **AgentCore と同じ VPC にデプロイ**するか、VPC ピアリングを設定
2. **サービスタイプを ClusterIP に変更するか、内部 Load Balancer を使用：**
   ```bash
   helm install openlit openlit/openlit \
     --set service.type=ClusterIP
   ```
   または AWS 内部ロードバランサーの場合：
   ```bash
   helm install openlit openlit/openlit \
     --set service.annotations."service\\.beta\\.kubernetes\\.io/aws-load-balancer-internal"="true"
   ```
3. **セキュリティグループ/ネットワークポリシーを設定**して AgentCore と OpenLIT 間のトラフィックを許可
4. 内部エンドポイントを使用（例：`http://openlit.default.svc.cluster.local:4318` または内部ロードバランサー DNS）

### OpenLIT エンドポイントの取得

OpenLIT がデプロイされたら、以下の形式で OTEL エンドポイントを使用します：
- **パブリック IP を持つ Docker**: `http://<ec2-public-ip-or-domain>:4318`
- **パブリック LoadBalancer を持つ Kubernetes**: `http://<load-balancer-external-ip>:4318`
- **内部/VPC を持つ Kubernetes**: `http://<internal-dns-or-ip>:4318`

**このエンドポイント URL を保存してください** - 以下の設定ステップで必要になります。

詳細なデプロイ手順については、[OpenLIT インストールガイド](https://docs.openlit.io/latest/openlit/installation)を参照してください。

## インストール

requirements.txt ファイルから必要な依存関係をインストールします：

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## エージェント実装

エージェントファイル（`strands_claude.py`）は、Web 検索機能を持つトラベルエージェントを実装しています。主な設定は以下を含みます：
- OTLP エクスポーターで Strands テレメトリを初期化
- 環境変数がロードされるように遅延初期化を使用

In [ ]:
%%writefile strands_claude.py
import os
import logging
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent, tool
from strands.models import BedrockModel
from strands.telemetry import StrandsTelemetry
from ddgs import DDGS

logging.basicConfig(level=logging.ERROR, format="[%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(os.getenv("AGENT_RUNTIME_LOG_LEVEL", "INFO").upper())


@tool
def web_search(query: str) -> str:
    """
    DuckDuckGo を使用して Web から情報を検索します。

    Args:
        query: 検索クエリ

    Returns:
        検索結果を含む文字列
    """
    try:
        ddgs = DDGS()
        results = ddgs.text(query, max_results=5)

        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(
                f"{i}. {result.get('title', 'No title')}\n"
                f"   {result.get('body', 'No summary')}\n"
                f"   Source: {result.get('href', 'No URL')}\n"
            )

        return "\n".join(formatted_results) if formatted_results else "No results found."

    except Exception as e:
        return f"Error searching the web: {str(e)}"

# Bedrock モデルを初期化する関数
def get_bedrock_model():
    region = os.getenv("AWS_DEFAULT_REGION", "us-west-2")
    model_id = os.getenv("BEDROCK_MODEL_ID", "us.anthropic.claude-3-7-sonnet-20250219-v1:0")

    bedrock_model = BedrockModel(
        model_id=model_id,
        region_name=region,
        temperature=0.0,
        max_tokens=1024
    )
    return bedrock_model

# Bedrock モデルを初期化
bedrock_model = get_bedrock_model()

# エージェントのシステムプロンプトを定義
system_prompt = """あなたはリアルタイムの Web 情報にアクセスできる、パーソナライズされた旅行推奨を専門とする経験豊富なトラベルエージェントです。
あなたの役割は、Web 検索を使用して最新情報を取得し、ユーザーの好みに合った理想の目的地を見つけることです。
最新情報、簡潔な説明、実用的な旅行の詳細を含む包括的な推奨を提供してください。"""

app = BedrockAgentCoreApp()

def initialize_agent():
    """適切なテレメトリ設定でエージェントを初期化します。"""

    # サードパーティ設定で Strands テレメトリを初期化
    strands_telemetry = StrandsTelemetry()
    strands_telemetry.setup_otlp_exporter()
    
    # エージェントを作成してキャッシュ
    agent = Agent(
        model=bedrock_model,
        system_prompt=system_prompt,
        tools=[web_search]
    )
    
    return agent

@app.entrypoint
def strands_agent_bedrock(payload, context=None):
    """
    ペイロードでエージェントを呼び出します
    """
    user_input = payload.get("prompt")
    logger.info("[%s] User input: %s", context.session_id, user_input)
    
    # 適切な設定でエージェントを初期化
    agent = initialize_agent()
    
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

### AgentCore Runtime デプロイの設定

次に、スターターツールキットを使用して、エントリポイント、作成した実行ロール、および requirements ファイルで AgentCore Runtime デプロイを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて Dockerfile が生成されます。`bedrock_agentcore_starter_toolkit` を使用してエージェントを設定する場合、デフォルトで AgentCore Observability が設定されることに注意してください。OpenLIT を使用するには、以下に説明するように AgentCore Observability の設定を削除する必要があります：

<div style="text-align:left">
    <img src="../images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_openlit_observability"
response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
    disable_otel=True,
)
response

## AgentCore Runtime へのデプロイ

Dockerfile ができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="../images/launch.png" width="75%"/>
</div>

In [ ]:
# OpenLIT 設定
# 重要：実際の OpenLIT OTEL エンドポイントに置き換えてください
# 例：
#   - パブリック EC2："http://ec2-XX-XXX-XXX-XXX.compute-1.amazonaws.com:4318"
#   - VPC/プライベート："http://10.0.1.100:4318" または "http://openlit.internal:4318"

otel_endpoint = "http://<your-openlit-host>:4318"  # ⚠️ OpenLIT エンドポイントに置き換えてください

env_vars = {
    "BEDROCK_MODEL_ID": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # モデル ID の例
    "OTEL_EXPORTER_OTLP_ENDPOINT": otel_endpoint,  # OpenLIT OTEL エンドポイントを使用
    "DISABLE_ADOT_OBSERVABILITY": "true",
}

launch_result = agentcore_runtime.launch(env_vars=env_vars)
launch_result

## デプロイステータスの確認

呼び出す前に、Runtime が準備完了になるまで待ちます：

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### AgentCore Runtime の呼び出し

最後に、ペイロードで AgentCore Runtime を呼び出すことができます。

<div style="text-align:left">
    <img src="../images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "I'm planning a weekend trip to Tokyo. What are the must-visit places and local food I should try?"})

In [ ]:
from IPython.display import Markdown, display
display(Markdown("".join(invoke_response['response'])))

## OpenLIT でトレースを表示

トレースを表示するには：
1. OpenLIT ダッシュボードにアクセス：
   - セルフホストの場合：`http://your-openlit-host:3000` に移動
2. 「Requests」セクションをクリック
3. サービス名でフィルタリング

トレースには以下が含まれます：
- 完全なリクエスト/レスポンスコンテキストを含むエージェント呼び出しの詳細
- 実行時間を含むツール呼び出し（Web 検索）
- レイテンシ、トークン使用量、コスト見積もりを含むモデルインタラクション
- リクエスト/レスポンスペイロード
- エラートラッキングとデバッグ情報
- パフォーマンスメトリクスと分析

## クリーンアップ（オプション）

デプロイしたリソースをクリーンアップ：

In [ ]:
import boto3

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

## まとめ

OpenLIT 可観測性を使用して Strands エージェントを Amazon Bedrock AgentCore Runtime に正常にデプロイしました。この実装は以下を示しています：
- Strands エージェントと AgentCore Runtime の統合
- トレースを OpenLIT に送信するための OpenTelemetry の設定
- テレメトリ設定を確実にするための適切な初期化順序
- SDK と boto3 クライアントの両方を通じた呼び出し

エージェントは、OpenLIT を通じた完全な可観測性を備えたマネージドでスケーラブルな環境で実行されています。OpenLIT は以下を含む包括的な監視を提供します：
- リアルタイムトレースの可視化
- コストトラッキングと分析
- パフォーマンスメトリクス
- エラートラッキングとデバッグ
- トークン使用量分析